In [1]:
import pandas as pd
import numpy as np
import torch
import json
from normalizer import counter, subSample
from torch.utils.data import DataLoader
from normalizer import subSample, getDataFromJSON

device = 'cpu' # Change this to cuda for GPU enabled computers

In [2]:
# This file has 77270 data points. 71633 do not have NAN values.
# %time train1, labels1, weights1 = getDataFromJSON(path="data/train_partition1_data.json", earlyStop=-1)
train1, labels1 = subSample("data/train_partition1_data.json", device=device)

Now loading event 1/785
Now loading event 101/785
Now loading event 201/785
Now loading event 301/785
Now loading event 401/785
Now loading event 501/785
Now loading event 601/785
Now loading event 701/785
785 lines loaded.


In [3]:
# This file has 93767 data points. 82425 of them do not have NAN values.  
# %time train2, labels2, weights2 = getDataFromJSON(path="data/train_partition2_data.json", earlyStop=-1)
train2, labels2 = subSample("data/train_partition2_data.json", device=device)

Now loading event 1/300
Now loading event 101/300
Now loading event 201/300
300 lines loaded.


In [4]:
# This file has 42986 data points. 37759 of them do not have NAN values.  
# %time train3, labels3, weights3 = getDataFromJSON(path="data/train_partition3_data.json", earlyStop=-1)
train3, labels3 = subSample("data/train_partition3_data.json", device=device)

Now loading event 1/585
Now loading event 101/585
Now loading event 201/585
Now loading event 301/585
Now loading event 401/585
Now loading event 501/585
585 lines loaded.


In [5]:
train2[0,1]

tensor([5.6095e+10, 5.6185e+10, 5.6320e+10, 5.6230e+10, 5.6633e+10, 5.6700e+10,
        5.6721e+10, 5.7005e+10, 5.7128e+10, 5.7335e+10, 5.7524e+10, 5.7700e+10,
        5.8026e+10, 5.8195e+10, 5.8293e+10, 5.8325e+10, 5.8458e+10, 5.8522e+10,
        5.8599e+10, 5.8633e+10, 5.8895e+10, 5.9083e+10, 5.9160e+10, 5.9369e+10,
        5.9312e+10, 5.9342e+10, 5.9696e+10, 5.9835e+10, 5.9866e+10, 6.0144e+10,
        6.0255e+10, 6.0501e+10, 6.0554e+10, 6.0493e+10, 6.0774e+10, 6.0800e+10,
        6.0813e+10, 6.0933e+10, 6.0885e+10, 6.1169e+10, 6.1346e+10, 6.1364e+10,
        6.1599e+10, 6.1777e+10, 6.1715e+10, 6.1813e+10, 6.1773e+10, 6.1688e+10,
        6.1471e+10, 6.1425e+10, 6.1285e+10, 6.1098e+10, 6.1073e+10, 6.0710e+10,
        6.0641e+10, 6.0710e+10, 6.0857e+10, 6.0728e+10, 6.0693e+10, 6.0833e+10])

In [6]:
train2[0,0]

tensor([3429.9802, 3444.6777, 3471.2712, 3477.1567, 3532.2834, 3470.6143,
        3478.2695, 3563.7605, 3509.6221, 3600.8823, 3586.3240, 3607.0430,
        3622.1245, 3583.2434, 3622.1433, 3645.5581, 3633.9895, 3682.0950,
        3606.4170, 3668.0813, 3689.6143, 3671.8491, 3660.6914, 3659.5105,
        3635.0242, 3653.2874, 3649.6904, 3647.1797, 3650.4246, 3620.0278,
        3671.7280, 3710.4827, 3690.0315, 3678.0454, 3646.0427, 3624.6687,
        3580.2256, 3626.6167, 3635.1226, 3618.3586, 3624.2644, 3622.0237,
        3640.3481, 3644.0481, 3658.2900, 3701.5437, 3687.8821, 3662.8955,
        3621.3562, 3651.4104, 3614.4851, 3563.1714, 3593.1545, 3526.1267,
        3511.8232, 3512.1658, 3475.2561, 3489.1379, 3482.3259, 3494.7253])

In [37]:
for X in [train1, train2, train3]:
    for i in range(X.shape[1]): # the number of types of measurements
        min_x = torch.min(X[:,i,:])
        X[:,i,:] -= min_x
        X[:,i,:] /= torch.max(X[:,i,:]).item()

In [39]:
train2[0,1]

tensor([0.4152, 0.4159, 0.4169, 0.4162, 0.4192, 0.4197, 0.4199, 0.4220, 0.4229,
        0.4244, 0.4258, 0.4271, 0.4295, 0.4308, 0.4315, 0.4317, 0.4327, 0.4332,
        0.4338, 0.4340, 0.4360, 0.4374, 0.4379, 0.4395, 0.4391, 0.4393, 0.4419,
        0.4429, 0.4432, 0.4452, 0.4460, 0.4479, 0.4483, 0.4478, 0.4499, 0.4501,
        0.4502, 0.4511, 0.4507, 0.4528, 0.4541, 0.4543, 0.4560, 0.4573, 0.4569,
        0.4576, 0.4573, 0.4567, 0.4551, 0.4547, 0.4537, 0.4523, 0.4521, 0.4494,
        0.4489, 0.4494, 0.4505, 0.4495, 0.4493, 0.4503])

In [67]:
# Define the network. Make sure to end with nn.Softmax activation
import torch.nn as nn
from skorch import NeuralNet

class logRegWithHidden(nn.Module):
    def __init__(self, hidden_size1, hidden_size2, num_classes=5, drop1=.5, input_size=1980):
        super().__init__() 
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.layerout = nn.Linear(hidden_size2, num_classes)
        #Define a RELU Activation unit
        self.relu = nn.ReLU()  
        self.smax = nn.Softmax(dim=1)
        self.drop = nn.Dropout(p=drop1)

    def forward(self, x):
        #Forward Propagate through the layers as defined above
        y = self.drop(x.reshape(-1, 1980))
        y = self.drop(self.relu(self.layer1(y)))
        y = self.relu(self.layer2(y))
        y = self.layerout(y)
        y = self.smax(y)
        return y



In [68]:
def train(modelModule, inputs, labels, weight, valSets, valLabels, valweight, *modelArgs, lr=0.01, **modelKwargs):
    # TODO: Is this right? How do I determine the weights here?
    
    #TODO: does this work?
    model = modelModule(*modelArgs, **modelKwargs)
    print(model)
    
    if weight is not None: weight = torch.Tensor(weight)
    lfc = nn.CrossEntropyLoss(weight=weight)
#     valLabels = torch.tensor(valLabels, dtype=torch.int)
    #ideas
    # 1-(weight/np.sum(weight))
    # .2/weight - this one normalizes so that each class is responsible for 20% of the loss
    # 1/weight - this is a bit naive, but the classes with fewer items are weighted more.
    # 1/(weight+1) - makes sure we don't have any pesky zeroes
    # np.sum(weight)/weight if your learning rate is too low.
    
    # Hyperparameters
    batch = 256
    epochs = 4 # artificially low for debugging
    
    # Start a dataloader object
    data = list(zip(inputs,labels))
    val = list(zip(valSets,valLabels))
    loader = DataLoader(data, batch_size = batch, num_workers=4)
    valLoader = DataLoader(val, batch_size = int(len(val)/4), num_workers=4)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    print(opt.state_dict())

    for epoch in range(epochs):
        model.train()
        batch_loss = []
        for (xtrain,ytrain) in loader:
            opt.zero_grad()
#             print(opt.state_dict())
            output = model(xtrain)
            loss = lfc(output,ytrain)
#             print(loss)
            loss.backward()
#             print(opt.state_dict())
#             print(model.state_dict())
            opt.step()
#             print(opt.state_dict())
#             print('\n\n\n\n\n')
            batch_loss.append(loss.item())
        print(f'The training loss for epoch {epoch+1}/{epochs} was {np.mean(batch_loss)}')
        
        
        model.eval()
        balanced = [[],[],[],[],[]]
        batchLoss = []
        unbalanced = []
        
        for (xval,yval) in valLoader:
            output = model(xval)
            loss = lfc(output,yval)
            batchLoss.append(loss.item())
            print(output)
            corrects = yval.clone().detach() == torch.argmax(output,1)
            print(yval.clone().detach(), torch.argmax(output,1))
            print(corrects.detach())
            print('===========================\n\n\n')
            unbalanced.append(np.mean([1 if correct else 0 for correct in corrects.detach()]))
            
            for i, ans in enumerate(yval):
                balanced[ans].append(corrects[i])
        
        balanced = [np.mean(i) for i in balanced]
        balancedAccuracy = np.mean(balanced)
        
#         print(f'The total balanced accuracy for validation was {balancedAccuracy}')
        print(f'The values of unbalanced are {unbalanced}')
        print(f'The validation loss was :   {epoch+1}/{epochs} was {np.mean(batchLoss)}')
        print(f'The unbalanced validation accuracy is {np.mean(unbalanced)}')
        print(f'The accuracy for each is {balanced}')           
    print(opt.state_dict())


    return model


In [69]:
# [weights1[i] + weights2[i] for i in range(5)]
trainArgs = [logRegWithHidden, torch.cat((train1, train2), dim=0), labels1 + labels2, None, train3, labels3, None]
modelArgs = [4096*4, 2048*4]
%time newModel = train(*trainArgs, *modelArgs, lr = 0.01, drop1=0)


logRegWithHidden(
  (layer1): Linear(in_features=1980, out_features=16384, bias=True)
  (layer2): Linear(in_features=16384, out_features=8192, bias=True)
  (layerout): Linear(in_features=8192, out_features=5, bias=True)
  (relu): ReLU()
  (smax): Softmax(dim=1)
  (drop): Dropout(p=0, inplace=False)
)
{'state': {}, 'param_groups': [{'lr': 0.01, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'params': [0, 1, 2, 3, 4, 5]}]}
torch.Size([256, 5])
torch.Size([256, 5])
torch.Size([256, 5])
torch.Size([256, 5])
torch.Size([61, 5])
The training loss for epoch 1/4 was 1.6428621768951417
torch.Size([146, 5])
tensor([[0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        

RuntimeError: The size of tensor a (146) must match the size of tensor b (5) at non-singleton dimension 0

In [33]:
s = 1 # choose this to check the sth layer
feature_extraction1 = [child for child in model.children()]
print(feature_extraction1[s])
feature_extraction2 = [child for child in newModel.children()]
print(feature_extraction2[s])
print(torch.max(feature_extraction1[s].weight - feature_extraction2[s].weight).detach())
print(torch.min(feature_extraction1[s].weight - feature_extraction2[s].weight).detach())

Linear(in_features=16384, out_features=8192, bias=True)
Linear(in_features=16384, out_features=8192, bias=True)
tensor(0.0156)
tensor(-0.0156)


In [ ]:
_ = train(
    model,
    torch.cat((train1, train3), dim=0),
    labels1 + labels3,
    None,
    train2,
    labels2,
    None,
    lr = 10
)

In [ ]:
_ = train(
    model,
    torch.cat((train3, train2), dim=0),
    labels3 + labels2,
    None,
    train1,
    labels1,
    None,
    lr = 0.00000001
)